<a href="https://colab.research.google.com/github/VerabelGonzales/Project_Alpha_Vision/blob/main/Stard_The_Search_For_The_PET_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install opencv-contrib-python==3.4.2.17

# Start the search for the PET

## Import Libreries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
from sklearn.cluster import KMeans
from collections import Counter
from skimage.color import rgb2lab, deltaE_cie76
import cv2
import os, requests
import copy
import pandas as pd
from statistics import mean
from numpy.ma.core import append
import glob
from IPython.display import Image, display
from numpy.ma.core import append
from statistics import mean
import copy

%matplotlib inline

## Clone our GitHub Repository

In [ ]:
!git clone https://github.com/VerabelGonzales/Project_Alpha_Vision

## Images provided by a person of a possible lost dog (LOST)

In [ ]:
def determine_breed(image_type):
  print(image_type)
  folders = ['Boxer','Chihuahua','German_Shepherd','Golden_Retriever','Pug','San_Bernardo','Schnauzer','Husky_Siveriano']
  breed = np.zeros(len(folders))
  for i in range(len(folders)):
    for imageName in glob.glob('/content/drive/MyDrive/Project_Alpha_Vision/'+image_type+'/'+folders[i]+'/*.jpg'): #assuming JPG
      breed[i] = breed[i] + 1
      display(Image(filename=imageName))
      print("\n")

  max_value = None
  max_idx = None
  for idx, num in enumerate(breed):
      if (max_value is None or num > max_value):
          max_value = num
          max_idx = idx
  
  return max_value, max_idx, folders[max_idx]

### Upload images of the possible lost dog 

In [ ]:
# The images are in a google drive folder 
for imageName in glob.glob('/content/drive/MyDrive/Images_of_a_lost_dog/*.jpg'):
    display(Image(filename=imageName)) # Show the images
    print("\n")

### Determine the breed of the possible lost dog (among our 8 selected breeds)

In [ ]:
# For a golden retriever
!python /content/Project_Alpha_Vision/Detection_Breed/detect_lost_golden.py --weights /content/drive/MyDrive/Project_Alpha_Vision/MODELS/golden.pt --img 416 --conf 0.1 --source  /content/drive/MyDrive/Images_of_a_lost_dog/

# For a Pug
!python /content/Project_Alpha_Vision/Detection_Breed/detect_lost_pug.py --weights /content/drive/MyDrive/Project_Alpha_Vision/MODELS/pug.pt --img 416 --conf 0.1 --source /content/drive/MyDrive/Images_of_a_lost_dog/

# For a German Shepherd 
#!python /content/Project_Alpha_Vision/Detection_Breed/Detect_Lost.py --weights /content/drive/MyDrive/2022_I/Artificial_Vision/Project_II/DOG_MODELS/golden.pt --img 416 --conf 0.1 --source /content/drive/MyDrive/golden_retriever/

# For a Chihuahua
#!python /content/Project_Alpha_Vision/Detection_Breed/Detect_Lost.py --weights /content/drive/MyDrive/2022_I/Artificial_Vision/Project_II/DOG_MODELS/golden.pt --img 416 --conf 0.1 --source /content/drive/MyDrive/golden_retriever/

# For a Husky Siveriano
#!python /content/Project_Alpha_Vision/Detection_Breed/Detect_Lost.py --weights /content/drive/MyDrive/2022_I/Artificial_Vision/Project_II/DOG_MODELS/golden.pt --img 416 --conf 0.1 --source /content/drive/MyDrive/golden_retriever/

#For a Boxer
#!python /content/Project_Alpha_Vision/Detection_Breed/Detect_Lost.py --weights /content/drive/MyDrive/2022_I/Artificial_Vision/Project_II/DOG_MODELS/golden.pt --img 416 --conf 0.1 --source /content/drive/MyDrive/golden_retriever/

# For a San Berbardo 
#!python /content/Project_Alpha_Vision/Detection_Breed/Detect_Lost.py --weights /content/drive/MyDrive/2022_I/Artificial_Vision/Project_II/DOG_MODELS/golden.pt --img 416 --conf 0.1 --source /content/drive/MyDrive/golden_retriever/

# For a Schauzer 
#!python /content/Project_Alpha_Vision/Detection_Breed/Detect_Lost.py --weights /content/drive/MyDrive/2022_I/Artificial_Vision/Project_II/DOG_MODELS/golden.pt --img 416 --conf 0.1 --source /content/drive/MyDrive/golden_retriever/


In [ ]:
images_type = 'LOST_DOGS'
dog_breed = determine_breed(images_type)

print('Maximum value:', dog_breed[0], "At index: ", dog_breed[1])
print('The breed of the lost dog is:',dog_breed[2])

### Determine the color of the dog 

In [ ]:
def RGB2HEX(color):
    return "#{:02x}{:02x}{:02x}".format(int(color[0]), int(color[1]), int(color[2]))

In [ ]:
def color_determination(imagename):
  image = cv2.imread(imagename)
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  number_of_colors = 1
  modified_image = image.reshape(image.shape[0]*image.shape[1], 3)
  clf = KMeans(n_clusters = number_of_colors)
  labels = clf.fit_predict(modified_image)

  counts = Counter(labels)

  center_colors = clf.cluster_centers_
  # We get ordered colors by iterating through the keys
  ordered_colors = [center_colors[i] for i in counts.keys()]
  hex_colors = [RGB2HEX(ordered_colors[i]) for i in counts.keys()]
  rgb_colors = [ordered_colors[i] for i in counts.keys()]

  #plt.title('Color Detection', fontsize=20)
  #plt.pie(counts.values(), labels = hex_colors, colors = hex_colors)

  return rgb_colors 

In [ ]:
def determine_color(image_type, n_images, breed):
  fils = int(n_images)
  colors = np.zeros((fils,3))
  cont = 0
  for imageName in glob.glob('/content/drive/MyDrive/Project_Alpha_Vision/'+image_type+'/'+breed+'/*.jpg'):
    colors[cont:cont+1] = color_determination(imageName)
    cont = cont + 1
  
  return colors 


In [ ]:
images_type = 'LOST_DOGS'
rgb_values = determine_color(images_type, dog_breed[0], dog_breed[2])


print(rgb_values)

RMX = max(rgb_values[0:int(dog_breed[0]),0])
GMX = max(rgb_values[0:int(dog_breed[0]),1])
BMX = max(rgb_values[0:int(dog_breed[0]),2])

RMN = min(rgb_values[0:int(dog_breed[0]),0])
GMN = min(rgb_values[0:int(dog_breed[0]),1])
BMN = min(rgb_values[0:int(dog_breed[0]),2])

R = mean(rgb_values[0:int(dog_breed[0]),0])
G = mean(rgb_values[0:int(dog_breed[0]),1])
B = mean(rgb_values[0:int(dog_breed[0]),2])

print('R',RMX)
print('G',GMX)
print('B',BMX)

print('R',RMN)
print('G',GMN)
print('B',BMN)

print('R',R)
print('G',G)
print('B',B)


### Seacrh by feature extraction

In [ ]:
def surf_feature_extraction(training_image, query_image):
  # Convert the training image to gray scale
  training_gray = cv2.cvtColor(training_image, cv2.COLOR_BGR2GRAY)
  # Convert the query image to gray scale
  query_gray = cv2.cvtColor(query_image, cv2.COLOR_BGR2GRAY)

  # Initiate SIFT detector
  surf = cv2.xfeatures2d.SURF_create()
  # find the keypoints and descriptors with SIFT
  keypoints_train, descriptors_train = surf.detectAndCompute(training_gray, None)
  keypoints_query, descriptors_query = surf.detectAndCompute(query_gray, None)

  # FLANN parameters
  FLANN_INDEX_KDTREE = 1
  index_params = dict (algorithm = FLANN_INDEX_KDTREE, trees = 5)
  search_params = dict (checks = 50)

  flann = cv2.FlannBasedMatcher(index_params,search_params)
  matches = flann.knnMatch(descriptors_train, descriptors_query, k = 2)

  # This value defines the minimum number of matches 
  Min_Match = 10

  #This vector store all the good matches
  good = []
  for m,n in matches:
    if m.distance < 0.5*n.distance:
      good.append(m)

    # We verify if we have enough coincidences (more than 10), if so we extract the locations of the key points that coincide in both images
    if len(good) > Min_Match:
      src_pts = np.float32([ keypoints_train[m.queryIdx].pt for m in good ]).reshape(-1,1,2)
      dst_pts = np.float32([ keypoints_query[m.trainIdx].pt for m in good ]).reshape(-1,1,2) 

      M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC,5.0) # Extract the locations of the key points that coincide in both images
      matchesMask = mask.ravel().tolist()

      h,w = training_gray.shape
      pts = np.float32([ [0,0],[0,h-1],[w-1,h-1],[w-1,0] ]).reshape(-1,1,2)
      dst = cv2.perspectiveTransform(pts,M) # We do a perspective transform 

      query_gray = cv2.polylines(query_gray,[np.int32(dst)],True,255,3, cv2.LINE_AA) # We use this OpenCV function to draw several lines at the same time 

    else:
      matchesMask = None
      
    draw_params = dict(matchColor = (0,255,255),singlePointColor = None, matchesMask = matchesMask,flags = 2)
    
  result = cv2.drawMatches(training_image, keypoints_train, query_image, keypoints_query, good, None, **draw_params)
  n_matching = len(good)
  
  return result, n_matching

In [ ]:
filenames = []
images = []

# The images are in a google drive folder 
for filename in os.listdir('/content/drive/MyDrive/Project_Alpha_Vision/LOST_DOGS/'+folders[max_idx]+'/'):
  filenames.append(filename)
  image = cv2.imread('/content/drive/MyDrive/Project_Alpha_Vision/LOST_DOGS/'+folders[max_idx]+'/'+filename)
  image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
  images.append(image)

num_images = len(images)
plt.figure(figsize=(80, 50))

# Students
for i in range(num_images):
    plt.subplot(1, num_images, i+1)
    index = i
    image = images[index]
    filename = filenames[index]
    plt.imshow(image)
    plt.title(filename)

In [ ]:
print('R',RMX)
print('G',GMX)
print('B',BMX)

print('R',RMN)
print('G',GMN)
print('B',BMN)

Class_image = cv2.imread('/content/drive/MyDrive/Project_Alpha_Vision/DATASET/WANTED DOGS/Golden_Retriever/2759564.jpg')
# Convert the training image to RGB
Class_image = cv2.cvtColor(Class_image, cv2.COLOR_BGR2RGB)
plt.figure(figsize = (20, 10))
plt.imshow(Class_image)
plt.title("")

In [ ]:
threshold = np.zeros(len(images))
for index in range(num_images):
  filename = filenames[index]
  result_surf = surf_feature_extraction(Class_image, images[index])
  plt.title(filename)
  plt.imshow(result_surf[0]),plt.show()
  threshold[index] = result_surf[1]